In [3]:
def load_and_show(schema, tables):
    df = None
    for table in tables:
        loop_df = spark.sql(
        f"""
        select '{table}' as tt,
        count(*) as cnt
        from {schema}.{table}
        """)
        if df is None:
            df = loop_df
        else:
            df = df.union(loop_df)
    df.show(truncate=False)

In [3]:
bronze_tables = [
    'campaign_events_raw',
    'customer_interactions_raw',
    'inventory_snapshots_raw',
    'marketing_campaigns_raw',
    'product_catalog_raw',
    'subscriptions_raw',
    'transaction_items_raw',
    'transactions_raw',
]
load_and_show('bronze', bronze_tables)

+--------------------+---+
|                  tt|cnt|
+--------------------+---+
| campaign_events_raw| 45|
|customer_interact...| 20|
|inventory_snapsho...| 40|
|marketing_campaig...| 15|
| product_catalog_raw| 20|
|   subscriptions_raw| 20|
|transaction_items...| 25|
|    transactions_raw| 20|
+--------------------+---+



In [2]:
silver_tables = [
'customer_interactions',
'inventory_snapshots',
'product_catalog',
'subscriptions',
'transaction_items',
'transactions',
]
load_and_show('silver', silver_tables)

25/10/24 14:31:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---------------------+---+
|tt                   |cnt|
+---------------------+---+
|customer_interactions|20 |
|inventory_snapshots  |40 |
|product_catalog      |20 |
|subscriptions        |20 |
|transaction_items    |25 |
|transactions         |20 |
+---------------------+---+



In [3]:
spark.sql('select * from silver.customer_interactions').show()

+--------------+-----------+---------------------+----------------+----------------+--------+--------+---------------+--------------------+---------------+-----------------------+-------------------+-----------+------------+-------------+--------------------+----------------+-----------------+
|interaction_id|customer_id|interaction_timestamp|interaction_date|interaction_type| channel|agent_id|       category|         subcategory|sentiment_score|resolution_time_minutes|satisfaction_rating|is_resolved|is_escalated|source_system|   created_timestamp|interaction_year|interaction_month|
+--------------+-----------+---------------------+----------------+----------------+--------+--------+---------------+--------------------+---------------+-----------------------+-------------------+-----------+------------+-------------+--------------------+----------------+-----------------+
|             7|      10004|  2025-10-20 12:40:18|      2025-10-20|    social_media| twitter|       5|      complai

In [5]:
spark.sql('show tables in gold').show(truncate=False)

+---------+--------------------------+-----------+
|namespace|tableName                 |isTemporary|
+---------+--------------------------+-----------+
|gold     |basket_analysis           |false      |
|gold     |campaign_roi_analysis     |false      |
|gold     |category_brand_performance|false      |
|gold     |channel_attribution       |false      |
|gold     |cohort_analysis           |false      |
|gold     |customer_360              |false      |
|gold     |product_performance       |false      |
|gold     |store_performance         |false      |
|gold     |subscription_health       |false      |
+---------+--------------------------+-----------+



In [4]:
gold_tables = [
'basket_analysis',
'campaign_roi_analysis',
'category_brand_performance',
'channel_attribution',
'cohort_analysis',
'customer_360',
'product_performance',
'store_performance',
'subscription_health',
]
load_and_show('gold', gold_tables)

25/10/24 15:54:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------------+---+
|tt                        |cnt|
+--------------------------+---+
|basket_analysis           |5  |
|campaign_roi_analysis     |10 |
|category_brand_performance|20 |
|channel_attribution       |20 |
|cohort_analysis           |1  |
|customer_360              |16 |
|product_performance       |20 |
|store_performance         |1  |
|subscription_health       |3  |
+--------------------------+---+

